In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [20]:
b3_stocks_1994_2020_path = "/Users/fabioleite/PycharmProjects/curso_spark_uepb/Bolsa_de_valores/arquivo_base/b3_stocks_1994_2020.csv"

In [21]:
b3_stocks = spark.read.csv(b3_stocks_1994_2020_path, header='true', inferSchema='true')

# Transformar data (datetime) para DD/MM/AAAA. Atualmente o formato está YYY-MM-DD. 1994-07-04
- Gerar um arquivo chamado b3stocks_T1.csv

In [22]:
b3_stocks.printSchema()

root
 |-- datetime: date (nullable = true)
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume: double (nullable = true)



In [34]:
from pyspark.sql.functions import to_date, col, date_format

# Tratar o campo datetime para o tipo correto de dados (to_datetime)
b3_stocks_dt_fromatada = b3_stocks.withColumn("data_formatada", date_format(b3_stocks.datetime, format="dd/MM/yyyy"))
# Criar um campo novo chamado data formatada com a data formatada no padrao solicitado
b3_stocks_dt_fromatada.show()

+----------+------+--------+--------+--------+--------+--------+--------------+
|  datetime|ticker|    open|   close|    high|     low|  volume|data_formatada|
+----------+------+--------+--------+--------+--------+--------+--------------+
|1994-07-04| ACE 3|    48.0|    48.0|    48.0|    47.0| 46550.0|    04/07/1994|
|1994-07-04| ALP 3|  155.27|   156.0|   156.0|  155.27|163405.8|    04/07/1994|
|1994-07-04| ALP 4|   131.0|   131.0|   131.0|   131.0|  6550.0|    04/07/1994|
|1994-07-04| IBP 6|   600.0|   600.0|   600.0|   600.0|  7800.0|    04/07/1994|
|1994-07-04| AQT 4|    0.89|    0.99|    0.99|    0.85| 13137.0|    04/07/1994|
|1994-07-04| ARN 4|145400.0|145400.0|145400.0|145400.0|  1454.0|    04/07/1994|
|1994-07-04| AVP 3|     2.9|     3.0|     3.0|     2.9|  1170.0|    04/07/1994|
|1994-07-04| BBR 3|    20.0|    20.0|    20.0|    20.0|  2000.0|    04/07/1994|
|1994-07-04| BAD 3|    13.0|    13.0|    13.0|    13.0|  3900.0|    04/07/1994|
|1994-07-04| BSE 4|     9.8|     9.8|   

In [44]:
quantidade_vazia = b3_stocks_dt_fromatada.filter(col("data_formatada").isNull()).count()
print(quantidade_vazia)  # Saída: 1

0


In [45]:
print(f"Datas vazias no df original: {b3_stocks_dt_fromatada.filter(col("data_formatada").isNull()).count()}")

Datas vazias no df original: 0


# T2 Considerando o arquivo anterior, fazer uma filtragem para que fique apenas um registro por dia. Esse registro deve ser apenas quele que possuir o maior volume negociado em bolsa.

In [ ]:
b3_stocks.info()

In [ ]:
b3_stocks.reset_index(inplace=True)

In [ ]:
# maior_valor_por_dia = b3_stocks.groupby(['datetime','ticker'])['volume'].max()
maior_valor_por_dia = b3_stocks.loc[b3_stocks.groupby('datetime')['volume'].idxmax()]
# maior_valor_por_dia.columns = ['datetime', 'ticker', 'volume']
# maior_valor_por_dia.reset_index(inplace=True)
# maior_valor_por_dia.to_csv("Bolsa_de_valores/transformacoes/maior_valor_por_dia.csv", header=True)
b3stocks_filtro_T2_path = 'Bolsa_de_valores/transformacoes/b3stocks_F1.csv'
b3stocks_filtro_T2 =  pd.read_csv(b3stocks_filtro_T2_path, header='infer')
b3stocks_filtro_T2['datetime'] = pd.to_datetime(b3stocks_filtro_T2['datetime'], format='%d/%m/%Y')

In [ ]:
# print(b3stocks_filtro_T2[['datetime','volume']])
# maior_valor_por_dia.columns.values[1] = 'volume'
# print(maior_valor_por_dia.columns)
# 
display(maior_valor_por_dia)

# print(b3stocks_filtro_T2[['datetime','volume']])

In [ ]:
# maior_valor_por_dia[['datetime','volume']].info()
b3stocks_filtro_T2[['datetime','volume']].info()

In [ ]:
maior_valor_por_dia.merge( b3stocks_filtro_T2, indicator=True, how='outer')
# pd.testing.assert_series_equal(maior_valor_por_dia[['datetime','volume']], b3stocks_filtro_T2[['datetime','volume']], check_names=False)

Considerando o arquivo da transformação T1 (b3stocks_T1.csv), fazer uma filtragem para que fique apenas os registros que possuírem volume negociado acima da média diária.

In [ ]:
# Calcula a média diária de volume
media_diaria = b3_stocks.groupby('datetime')['volume'].mean()

# Cria uma coluna com a média diária
b3_stocks['media_diaria'] = b3_stocks['datetime'].map(media_diaria)
b3_stocks['media_diaria'] = b3_stocks['media_diaria'].apply(float)
# Filtra as linhas com volume acima da média diária
b3stocks_Media_diaria_maior = b3_stocks[b3_stocks['volume'] > b3_stocks['media_diaria']]

In [ ]:
b3stocks_Media_diaria_maior.info()

In [ ]:
b3_stocks['media_diaria'] = b3_stocks['media_diaria'].map('{:.2f}'.format)

In [ ]:
display(b3stocks_Media_diaria_maior.sort_values('datetime', ascending=True))

In [ ]:
b3stocks_Filtro_T3_path = "Bolsa_de_valores/transformacoes/b3stocks_T1_Filtrado.csv"
b3stocks_filtro_T3 = pd.read_csv(b3stocks_Filtro_T3_path, header='infer')
b3stocks_filtro_T3['datetime'] = pd.to_datetime(b3stocks_filtro_T3['datetime'], format='%d/%m/%Y')


In [ ]:
resultado = b3stocks_Media_diaria_maior.merge(b3stocks_filtro_T3, indicator=True, how='outer')

In [ ]:
print("Total de registros filtrados: {:,}".format(len(b3stocks_Media_diaria_maior)))
print("Total de registros filtrados pelo grupo: {:,}".format(len(b3stocks_filtro_T3)))
print("Total de registros filtrados diferença: {:,}".format(len(resultado[resultado['_merge'] != 'both'])))


In [ ]:
# Remove a coluna auxiliar
# df_filtrado = df_filtrado.drop('media_diaria', axis=1)